In [ ]:
import os
# import imp
import sklearn.preprocessing as skp
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import librosa.display
import librosa
import pandas as pd
import os
from matplotlib import cm, colors, pyplot as plt

import IPython 
from IPython.display import display

import numpy as np
import pandas as pd

import librosa 
import librosa.display

%matplotlib inline
from sklearn.preprocessing import minmax_scale

In [ ]:
# class Config:
#     tracks = 100
#     segments = 10
#     seconds = 3
#     rate = 22050
#     data_dir = 'data'
#     lib_dir = 'D:\\Musik\\Genres\\'
#     exc_dir = ['DnB', 'Trash', 'Other', 'JumpUp']
#     sub_dir = os.path.join(data_dir, f'{tracks}x{segments}x{seconds}')
# config = Config()

In [ ]:
class Config:
    tracks = 1
    segments = 100
    seconds = 3
    rate = 22050
    data_dir = 'data'
    lib_dir = 'D:\\Musik\\Genres\\'
    exc_dir = ['DnB', 'Trash', 'Other', 'JumpUp']
    sub_dir = os.path.join(data_dir, f'{tracks}x{segments}x{seconds}')
config = Config()

In [ ]:
from src.Database import Database

db = Database(config.sub_dir)
print("Dataset has",db.df.shape)
print("Count of Positive and Negative samples")
db.df.label.value_counts().reset_index()



In [ ]:
# SELECTED = ['Jazz', 'Classical', 'Hiphop', 'Disco', 'Country', 'Blues']
# db.df = db.df.loc[db.df['label'].isin(SELECTED)]
df = db.df
print("Dataset has",df.shape)
print("Count of Positive and Negative samples")
df.reset_index()

In [ ]:
# Computing the Correlation Matrix
spike_cols = [col for col in df.columns if 'mean' in col]
corr = df[spike_cols].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(16, 11));

# Generate a custom diverging colormap
cmap = sns.diverging_palette(0, 25, as_cmap=True, s = 90, l = 45, n = 5)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Correlation Heatmap (for the MEAN variables)', fontsize = 20)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.savefig(os.path.join(config.sub_dir, "Corr_Heatmap.png"))

In [ ]:
x = df[["label", "tempo"]]

fig, ax = plt.subplots(figsize=(16, 8));
sns.boxplot(x = "label", y = "tempo", data = x, palette = 'husl');

plt.title('BPM Boxplot for Genres', fontsize = 20)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 10)
plt.xlabel("Genre", fontsize = 15)
plt.ylabel("BPM", fontsize = 15)
plt.savefig(os.path.join(config.sub_dir, "BPM_Boxplot.png"))

In [ ]:
data = df.iloc[0:, 1:]
y = data['label']
X = data.loc[:, data.columns != 'label']

# normalize
cols = X.columns
min_max_scaler = skp.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(np_scaled, columns = cols)

# Top 2 pca components
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents, columns = ['pc1', 'pc2'])

# concatenate with target label
finalDf = pd.concat([principalDf, y], axis = 1)

plt.figure(figsize = (16, 9))
sns.scatterplot(x = "pc1", y = "pc2", data = finalDf, hue = "label", alpha = 0.7, s = 100);

plt.title('PCA on Genres', fontsize = 20)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 10);
plt.xlabel("Principal Component 1", fontsize = 15)
plt.ylabel("Principal Component 2", fontsize = 15)
plt.savefig("PCA_Scattert.png")